# Create a pipeline

This notebook demonstrates how to use the SDK to create pipelines.

We begin by creating a client with our Personal Access Token.

In [ ]:
from glassflow import GlassFlowClient

# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile
personal_access_token = "< your-personal-access-token >"

In [ ]:
client = GlassFlowClient(
    personal_access_token=personal_access_token
)

The next step is to create a transformation function to run on the pipeline. Let's create a simple transformation that prints the event data to the logs.

In [ ]:
%%bash
echo "Writing transformation function ..."
cat <<EOT > transformation.py 
def handler(data, log):
    log.info(data)
    return data
EOT
echo "Done!"

In GlassFlow, pipelines need to be created inside a space.

In [ ]:
space = client.create_space(name="tutorials")
print("Space ID:", space.id)

Now, we have everything we need to create a pipeline:

In [ ]:
pipeline = client.create_pipeline(
    name="example pipeline",
    space_id=space.id,
    transformation_file="transformation.py",
)
print("Pipeline ID:", pipeline.id)

In [ ]:
from IPython.display import Markdown as md

md(f"```python\n{pipeline.transformation_code}\n```")

If we look at the pipeline logs, we can see that the pipeline was created successfully! 🚀

(Logs might take a few seconds to appear, so you may need to run this cell a few times for the logs to show up.)

In [ ]:
logs_res = pipeline.get_logs()
for log in logs_res.logs:
    print(log.to_json())

### Transformation Requirements

The previous example demonstrates a basic pipeline. In some cases, we might need to include additional dependencies for our transformation code. We can pass a requirements.txt file to the pipeline.

In [ ]:
pipeline_with_requirements = client.create_pipeline(
    name="example pipeline with requirements",
    transformation_file="transformation.py",
    space_id=space.id,
    requirements="requests==2.32.3"
)
print("Pipeline ID:", pipeline_with_requirements.id)

### Environment Variables

Pipelines support environment variables

In [ ]:
pipeline_with_env_vars = client.create_pipeline(
    name="example pipeline with requirements",
    transformation_file="transformation.py",
    space_id=space.id,
    env_vars=[
        {
            "name": "MY_VAR",
            "value": "example value"
        }
    ]
)
print("Pipeline ID:", pipeline_with_env_vars.id)

## Clean-up

Delete the pipelines and spaces we created:

In [ ]:
pipeline.delete()
pipeline_with_env_vars.delete()
pipeline_with_requirements.delete()
space.delete()